# Endpoints

In [6]:
import yaml
from solrscope import aa
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import requests
from importlib import reload 
from solrscope import client_solr
from solrscope import client_ambit
from solrscope import annotation
import pandas as pd
import numpy as np
import json
import pandas as pd
import qgrid
import warnings
warnings.simplefilter("ignore")

In [7]:
print('Select enanoMapper aggregated search service:')
style = {'description_width': 'initial'}
config,config_servers, config_security, auth_object, msg = aa.parseOpenAPI3()    
service_widget = widgets.Dropdown(
    options=config_servers['url'],
    description='Service:',
    disabled=False,
    style=style
)
if config_security is None:
    service = interactive(aa.search_service_open,url=service_widget)
else:
    print(msg)
    apikey_widget=widgets.Text(
            placeholder='',
            description=config_security,
            disabled=False,
            style=style
    )    
    service = interactive(aa.search_service_protected,url=service_widget,apikey=apikey_widget)    

display(service)

Select enanoMapper aggregated search service:
Enter `X-Gravitee-Api-Key` you have received upon subscription to http://api.ideaconsult.net


interactive(children=(Dropdown(description='Service:', options=('https://api.ideaconsult.net/enanomapper', 'ht…

In [20]:
service_uri=service_widget.value
if auth_object!=None:
    auth_object.setKey(apikey_widget.value)
print("Sending queries to {}".format(service_uri))



Sending queries to https://api.ideaconsult.net/nanoreg1


In [21]:
query_category="ASPECT_RATIO_SHAPE_SECTION"
query_endpoint='ASPECT'
data = {'fq': 'type_s:study', 'q': 'endpointcategory_s:'+query_category, 'wt':'json', 'indent':'on', 'rows': 1000000}
results = client_solr.post(service_uri,query=data,auth=auth_object)    
response_json=results.json()

In [36]:
from pandas.io.json import json_normalize
df=json_normalize( response_json["response"]["docs"])
df.fillna("",inplace=True)
#print(df.columns)
#df = df[df['effectendpoint_s'].str.contains(query_endpoint)]
try:
    df['loValue_d'] = pd.to_numeric(df['loValue_d'],errors='coerce')
except Exception as e:
    pass
try:
    df['upValue_d'] = pd.to_numeric(df['upValue_d'],errors='coerce')
except Exception as e:
    pass
try:
    df['err_d'] = pd.to_numeric(df['upValue_d'],errors='coerce')
except Exception as e:
    pass
try:
    a = annotation.DictionarySubstancetypes()
    df[ 'substanceType_name']=df[ 'substanceType_s'].apply(a.annotate)    
except Exception as e:
    pass
if "endpointcategory_s" in df.columns:    
    a = annotation.DictionaryEndpointCategory()
    df[ 'endpointcategory_term']=df[ 'endpointcategory_s'].apply(a.annotate)
    a = annotation.DictionaryEndpointCategoryNames()
    df[ 'endpointcategory_name']=df[ 'endpointcategory_s'].apply(a.annotate)
#df.info()

In [38]:
cols=[]
for col in ["owner_name_s","substanceType_name","publicname_s","endpointcategory_name","guidance_s","reference_s","reference_owner_s","effectendpoint_s","effectendpoint_type_s","loQualifier_s","loValue_d","upQualifier_s","upValue_d","err_qualifier_s","err_d","unit_s","textValue_s"]:
    if col in df.columns:
        cols.append(col)        
display(df[cols].head())


,owner_name_s,substanceType_name,publicname_s,endpointcategory_name,guidance_s,reference_s,reference_owner_s,effectendpoint_s,effectendpoint_type_s,loValue_d,err_d,unit_s
0,NANoREG,titanium dioxide,JRCNM01000a,Aspect ratio/shape,TEM,ILC_Phase3_CODA_PP,CODA-CERVA,ASPECT RATIO,MEDIAN,0.80,0.02,
1,NANoREG,titanium dioxide,JRCNM01000a,Aspect ratio/shape,TEM,ILC_Phase3_CODA_PP,CODA-CERVA,ASPECT RATIO,MEAN,0.77,0.02,
2,NANoREG,titanium dioxide,JRCNM01000a,Aspect ratio/shape,TEM,ILC_Phase3_CODA_PP,CODA-CERVA,ASPECT RATIO,MODE,0.86,0.03,
3,NANoREG,titanium dioxide,JRCNM01000a,Aspect ratio/shape,TEM,ILC_Phase3_CODA_PP,CODA-CERVA,CIRCULARITY,MEDIAN,0.82,0.02,
4,NANoREG,titanium dioxide,JRCNM01000a,Aspect ratio/shape,TEM,ILC_Phase3_CODA_PP,CODA-CERVA,CIRCULARITY,MEAN,0.80,0.02,


In [39]:
table = pd.pivot_table(df, values='loValue_d', columns=['effectendpoint_s','effectendpoint_type_s','reference_owner_s'], index=["publicname_s"], aggfunc=np.mean).fillna("").reset_index()
table

effectendpoint_s                    publicname_s ASPECT RATIO               \
effectendpoint_type_s                                    MEAN                
reference_owner_s                                  CODA-CERVA   INL   NMBU   
0                                    JRCNM01000a        0.715                
1                                    JRCNM01003a        0.615                
2                                    JRCNM02000a       0.6175                
3                                    JRCNM02102a        0.655                
4                                        NM-300K         0.88        1.115   
5                      NM-300K dispersion medium               0.86          

effectendpoint_s                                                       ...  \
effectendpoint_type_s     MEDIAN                    MODE               ...   
reference_owner_s     CODA-CERVA   INL   NMBU CODA-CERVA   INL   NMBU  ...   
0                          0.755                   0.875               ...   
1                           0.65                   0.735               ...   
2                          0.665                   0.745               ...   
3                          0.695                     0.8               ...   
4                          0.895        1.125      0.915        1.085  ...   
5                                 0.88                    0.88         ...   

effectendpoint_s      ROUNDNESS   SOLIDITY                                \
effectendpoint_type_s      MODE       MEAN                  MEDIAN         
reference_owner_s          NMBU CODA-CERVA   INL   NMBU CODA-CERVA   INL   
0                                    0.935                   0.945         
1                                     0.87                    0.88         
2                                     0.83                    0.84         
3                                    0.865                    0.87         
4                         0.785       0.95        0.935      0.955         
5                                           0.86                    0.85   

effectendpoint_s                                      
effectendpoint_type_s              MODE               
reference_owner_s       NMBU CODA-CERVA   INL   NMBU  
0                                 0.915               
1                                   0.9               
2                                  0.83               
3                                 0.875               
4                      0.935       0.96        0.935  
5                                        0.88         

[6 rows x 37 columns]